In [82]:
import numpy as np
import torch
import torchvision
import torch.nn as nn
from torchvision import datasets 
import matplotlib.pyplot as plt
import pandas as pd
import os
from torch.utils.data import DataLoader, Dataset

In [73]:
train_data_path = os.path.join('../../data', 'sign_language_mnist', 'sign_mnist_train.csv')

In [138]:
train_data = pd.read_csv(train_data_path)
print(train_data['label'].value_counts())
train_data.head()

17    1294
16    1279
11    1241
22    1225
5     1204
18    1199
3     1196
14    1196
19    1186
23    1164
8     1162
20    1161
13    1151
2     1144
0     1126
24    1118
10    1114
6     1090
15    1088
21    1082
12    1055
7     1013
1     1010
4      957
Name: label, dtype: int64


,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179


In [139]:
class PictureDataset(Dataset):
    def __init__(self, train_data):
        self.training_labels = torch.tensor(train_data['label'].values)
        self.training_features = torch.tensor(train_data.drop(columns=['label']).values).reshape(27455, 28, 28)
        
    def __len__(self):
        return len(self.training_labels)
        
    def __getitem__(self, idx):
        return self.training_features[idx], self.training_labels[idx]

In [140]:
train_dataloader = DataLoader(PictureDataset(train_data), batch_size=64, shuffle=True)

In [145]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(784, 250)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(250, 250)
        self.fc3 = nn.Linear(250, 25)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

In [146]:
model = NeuralNetwork()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [147]:
num_epochs = 250

total_step = len(train_dataloader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_dataloader):  
        images = images.reshape(-1, 28*28)
        labels = labels
        
        outputs = model(images.float())
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/50], Step [100/429], Loss: 3.1589
Epoch [1/50], Step [200/429], Loss: 3.1667
Epoch [1/50], Step [300/429], Loss: 3.1782
Epoch [1/50], Step [400/429], Loss: 3.1595
Epoch [2/50], Step [100/429], Loss: 3.1655
Epoch [2/50], Step [200/429], Loss: 3.1690
Epoch [2/50], Step [300/429], Loss: 3.1698
Epoch [2/50], Step [400/429], Loss: 3.1842
Epoch [3/50], Step [100/429], Loss: 3.1858
Epoch [3/50], Step [200/429], Loss: 3.1894
Epoch [3/50], Step [300/429], Loss: 3.1651
Epoch [3/50], Step [400/429], Loss: 3.1727
Epoch [4/50], Step [100/429], Loss: 3.1516
Epoch [4/50], Step [200/429], Loss: 3.1934
Epoch [4/50], Step [300/429], Loss: 3.1661
Epoch [4/50], Step [400/429], Loss: 3.1847
Epoch [5/50], Step [100/429], Loss: 3.2135
Epoch [5/50], Step [200/429], Loss: 3.1625
Epoch [5/50], Step [300/429], Loss: 3.1479
Epoch [5/50], Step [400/429], Loss: 3.1713
Epoch [6/50], Step [100/429], Loss: 3.1892
Epoch [6/50], Step [200/429], Loss: 3.2013
Epoch [6/50], Step [300/429], Loss: 3.1733
Epoch [6/50

KeyboardInterrupt: 

In [117]:
class PictureDataset(Dataset):
    def __init__(self, train_data):
        self.training_labels = torch.tensor(train_data['label'].values)
        self.training_features = torch.tensor(train_data.drop(columns=['label']).values).reshape(len(self.training_labels), 28, 28)
        
    def __len__(self):
        return len(self.training_labels)
        
    def __getitem__(self, idx):
        return self.training_features[idx], self.training_labels[idx]

In [118]:
test_data_path = os.path.join('../../data', 'sign_language_mnist', 'sign_mnist_test.csv')
test_data = pd.read_csv(test_data_path)
test_loader = DataLoader(PictureDataset(test_data), batch_size=64, shuffle=False)

In [126]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        labels = labels
        outputs = model(images.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 2.007808142777468 %
